In [1]:
import pathlib
import polars as pl

In [2]:
path_data = pathlib.Path().cwd().parent / "data" 

df = pl.read_csv(path_data / 'train.csv')

In [3]:
df.shape

(8000, 17)

In [4]:
df.describe()

statistic,Customer_ID,Age,Gender,Location,Subscription_Type,Account_Age_Months,Monthly_Spending,Total_Usage_Hours,Support_Calls,Late_Payments,Streaming_Usage,Discount_Used,Satisfaction_Score,Last_Interaction_Type,Complaint_Tickets,Promo_Opted_In,Churn
str,f64,f64,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64
"""count""",8000.0,8000.0,"""8000""","""8000""","""8000""",8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,"""8000""",8000.0,8000.0,8000.0
"""null_count""",0.0,0.0,"""0""","""0""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""0""",0.0,0.0,0.0
"""mean""",5000.5,43.54225,null,null,null,30.163875,104.804641,254.326625,4.45325,1.9925,49.798,49.42825,5.462375,null,1.9705,0.49975,0.313125
"""std""",2309.54541,14.909242,null,null,null,16.942407,54.643255,140.855632,2.88892,1.39971,28.965468,28.662071,2.879865,null,1.413375,0.500031,0.463794
"""min""",1001.0,18.0,"""Female""","""California""","""Basic""",1.0,10.09,10.0,0.0,0.0,0.0,0.0,1.0,"""Negative""",0.0,0.0,0.0
"""25%""",3001.0,31.0,null,null,null,15.0,57.64,133.0,2.0,1.0,24.0,25.0,3.0,null,1.0,0.0,0.0
"""50%""",5001.0,44.0,null,null,null,30.0,104.71,257.0,4.0,2.0,51.0,50.0,5.0,null,2.0,0.0,0.0
"""75%""",7000.0,57.0,null,null,null,45.0,151.69,376.0,7.0,3.0,75.0,74.0,8.0,null,3.0,1.0,1.0
"""max""",9000.0,69.0,"""Male""","""Texas""","""Premium""",59.0,199.94,499.0,9.0,4.0,99.0,99.0,10.0,"""Positive""",4.0,1.0,1.0


In [5]:
# Variáveis demográficas
demographic_vars = ['Age', 'Gender', 'Location']

# Variáveis categóricas
categorical_vars = [
    'Gender',
    'Location', 
    'Subscription_Type',
    'Last_Interaction_Type'
]

# Variáveis binárias (podem ser tratadas como categóricas ou numéricas)
binary_vars = [
    'Promo_Opted_In',  # 0/1
    'Churn'            # 0/1 - Target variable
]

# Variáveis numéricas contínuas
continuous_vars = [
    'Age',
    'Account_Age_Months',
    'Monthly_Spending',
    'Total_Usage_Hours',
    'Streaming_Usage',     # Porcentagem (0-99%)
    'Discount_Used',       # Porcentagem (0-99%)
    'Satisfaction_Score'   # Escala 1-10
]

# Variáveis numéricas discretas (contagens)
discrete_vars = [
    'Support_Calls',       # Número de chamadas
    'Late_Payments',       # Número de pagamentos atrasados
    'Complaint_Tickets'    # Número de tickets de reclamação
]

# Todas as variáveis numéricas (contínuas + discretas + binárias)
numeric_vars = continuous_vars + discrete_vars + binary_vars

# Variável alvo
target_var = 'Churn'

# Variáveis explicativas (features)
feature_vars = [col for col in df.columns if col not in ['Customer_ID', 'Churn']]

In [6]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from typing import List, Optional

# Configuração de estilo
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [11]:
import os
import polars as pl

# ==> força backend não interativo ANTES de importar pyplot
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

from explor_plots_funcs import (
    plot_target_distribution,
    plot_categorical_analysis,
    plot_numerical_distributions,
    plot_boxplots_by_target,
    plot_correlation_matrix,
    plot_pairplot_sample
)

def _save_matplotlib_figure(fig, path_png: str, path_pdf: str | None = None, dpi: int = 150):
    try:
        fig.tight_layout()
    except Exception:
        pass
    fig.savefig(path_png, bbox_inches="tight", dpi=dpi)
    if path_pdf:
        fig.savefig(path_pdf, bbox_inches="tight")
    plt.close(fig)

def _try_save_returned_object(obj, basename: str, output_dir: str) -> bool:
    """Tenta salvar baseado no objeto retornado. Retorna True se salvou algo."""
    if obj is None:
        return False

    # 1) matplotlib Figure (tem savefig)
    if hasattr(obj, "savefig"):
        _save_matplotlib_figure(obj, os.path.join(output_dir, f"{basename}.png"),
                                     os.path.join(output_dir, f"{basename}.pdf"))
        return True

    # 2) matplotlib Axes
    if hasattr(obj, "get_figure"):
        try:
            fig = obj.get_figure()
            if fig is not None:
                _save_matplotlib_figure(fig, os.path.join(output_dir, f"{basename}.png"),
                                             os.path.join(output_dir, f"{basename}.pdf"))
                return True
        except Exception:
            pass

    # 3) seaborn FacetGrid/PairGrid (.fig)
    if hasattr(obj, "fig"):
        fig = getattr(obj, "fig", None)
        if fig is not None:
            _save_matplotlib_figure(fig, os.path.join(output_dir, f"{basename}.png"),
                                         os.path.join(output_dir, f"{basename}.pdf"))
            return True

    # 4) objetos com .figure
    if hasattr(obj, "figure"):
        fig = getattr(obj, "figure", None)
        if fig is not None:
            _save_matplotlib_figure(fig, os.path.join(output_dir, f"{basename}.png"),
                                         os.path.join(output_dir, f"{basename}.pdf"))
            return True

    # 5) Plotly Figure
    try:
        import plotly.graph_objs as go  # noqa
        if hasattr(obj, "to_image") or hasattr(obj, "write_image"):
            # write_image é preferível (precisa kaleido instalado)
            png_path = os.path.join(output_dir, f"{basename}.png")
            pdf_path = os.path.join(output_dir, f"{basename}.pdf")
            if hasattr(obj, "write_image"):
                obj.write_image(png_path, scale=2)
                obj.write_image(pdf_path)
            else:
                # fallback com to_image
                png_bytes = obj.to_image(format="png", scale=2)
                with open(png_path, "wb") as f:
                    f.write(png_bytes)
            return True
    except Exception:
        pass

    return False

def _run_and_save(func, fname_base: str, output_dir: str, *args, **kwargs):
    """
    Executa uma função de plot, tenta salvar objeto retornado;
    se nada for salvo, salva TODAS as novas figuras matplotlib criadas.
    """
    # Snapshot das figuras abertas antes
    before = set(plt.get_fignums())

    # Executa a função e tenta salvar o retorno
    ret = func(*args, **kwargs)
    saved_any = _try_save_returned_object(ret, fname_base, output_dir)

    # Se não salvou via retorno, tenta capturar novas figuras criadas
    after = set(plt.get_fignums())
    new_fig_nums = sorted(list(after - before))

    if not saved_any and new_fig_nums:
        # Se criou mais de uma figura, salva enumerando
        if len(new_fig_nums) == 1:
            fig = plt.figure(new_fig_nums[0])
            _save_matplotlib_figure(
                fig,
                os.path.join(output_dir, f"{fname_base}.png"),
                os.path.join(output_dir, f"{fname_base}.pdf"),
            )
            saved_any = True
        else:
            for i, num in enumerate(new_fig_nums, start=1):
                fig = plt.figure(num)
                _save_matplotlib_figure(
                    fig,
                    os.path.join(output_dir, f"{fname_base}_{i:02d}.png"),
                    os.path.join(output_dir, f"{fname_base}_{i:02d}.pdf"),
                )
            saved_any = True

    # Último fallback: gcf (evita branco se não houve nada de novo)
    if not saved_any:
        try:
            fig = plt.gcf()
            _save_matplotlib_figure(
                fig,
                os.path.join(output_dir, f"{fname_base}.png"),
                os.path.join(output_dir, f"{fname_base}.pdf"),
            )
            saved_any = True
        except Exception:
            pass

    return saved_any

def comprehensive_eda(
    df: pl.DataFrame, 
    categorical_vars: list[str], 
    numeric_vars: list[str],
    output_dir: str = "eda_plots"
):
    """Executa análise exploratória completa e salva os gráficos (PNG e PDF)."""
    print("=== ANÁLISE EXPLORATÓRIA DE DADOS ===\n")
    os.makedirs(output_dir, exist_ok=True)

    # 1. Distribuição do target
    print("1. Distribuição da Variável Target")
    _run_and_save(plot_target_distribution, "01_target_distribution", output_dir, df)

    # 2. Análise das variáveis categóricas
    print("\n2. Análise das Variáveis Categóricas")
    _run_and_save(plot_categorical_analysis, "02_categorical_analysis", output_dir, df, categorical_vars)

    # 3. Distribuições numéricas
    print("\n3. Distribuições das Variáveis Numéricas")
    _run_and_save(plot_numerical_distributions, "03_numeric_distributions", output_dir, df, numeric_vars)

    # 4. Boxplots por Target
    print("\n4. Boxplots por Target")
    _run_and_save(plot_boxplots_by_target, "04_boxplots_by_target", output_dir, df, numeric_vars)

    # 5. Matriz de correlação
    print("\n5. Matriz de Correlação")
    _run_and_save(plot_correlation_matrix, "05_correlation_matrix", output_dir, df, numeric_vars)

    # 6. Pairplot de variáveis selecionadas
    print("\n6. Pairplot - Variáveis Importantes")
    important_vars = ['Age', 'Monthly_Spending', 'Satisfaction_Score', 'Support_Calls', 'Late_Payments']
    _run_and_save(plot_pairplot_sample, "06_pairplot", output_dir, df, important_vars)

    print(f"\n=== ANÁLISE CONCLUÍDA ===\nPlots salvos em: {os.path.abspath(output_dir)}")

In [12]:
import missingno as msno

# Assuming 'df' is your DataFrame
msno.matrix(df.to_pandas())


<Axes: >

In [13]:
df.to_pandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Customer_ID            8000 non-null   int64  
 1   Age                    8000 non-null   int64  
 2   Gender                 8000 non-null   object 
 3   Location               8000 non-null   object 
 4   Subscription_Type      8000 non-null   object 
 5   Account_Age_Months     8000 non-null   int64  
 6   Monthly_Spending       8000 non-null   float64
 7   Total_Usage_Hours      8000 non-null   int64  
 8   Support_Calls          8000 non-null   int64  
 9   Late_Payments          8000 non-null   int64  
 10  Streaming_Usage        8000 non-null   int64  
 11  Discount_Used          8000 non-null   int64  
 12  Satisfaction_Score     8000 non-null   int64  
 13  Last_Interaction_Type  8000 non-null   object 
 14  Complaint_Tickets      8000 non-null   int64  
 15  Prom

In [14]:
# Executar análise completa
path_plots = pathlib.Path().cwd() / "plots_eda"

comprehensive_eda(df, output_dir=path_plots, categorical_vars=categorical_vars, numeric_vars=numeric_vars)

=== ANÁLISE EXPLORATÓRIA DE DADOS ===

1. Distribuição da Variável Target
Taxa de Churn: 31.31%
Total de clientes: 8000
Clientes que fizeram churn: 2505
Clientes que permaneceram: 5495


c:\Users\leand\Documents\GitHub\curstomer-churn-prediction-kaggle\notebooks\explor_plots_funcs.py:33: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()



2. Análise das Variáveis Categóricas


c:\Users\leand\Documents\GitHub\curstomer-churn-prediction-kaggle\notebooks\explor_plots_funcs.py:81: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()



3. Distribuições das Variáveis Numéricas


...


5. Matriz de Correlação

Top 10 correlações com Churn:
Churn                 1.000000
Discount_Used         0.020400
Satisfaction_Score    0.019819
Promo_Opted_In        0.017182
Late_Payments         0.016602
Age                   0.016458
Support_Calls         0.012913
Complaint_Tickets     0.010890
Monthly_Spending      0.009202
Streaming_Usage       0.008729
Name: Churn, dtype: float64


c:\Users\leand\Documents\GitHub\curstomer-churn-prediction-kaggle\notebooks\explor_plots_funcs.py:149: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()



6. Pairplot - Variáveis Importantes


c:\Users\leand\Documents\GitHub\curstomer-churn-prediction-kaggle\notebooks\explor_plots_funcs.py:205: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()



=== ANÁLISE CONCLUÍDA ===
Plots salvos em: c:\Users\leand\Documents\GitHub\curstomer-churn-prediction-kaggle\notebooks\plots_eda


### Conclusões da analise exploratoria

1. O dataset não tem dados faltantes, então para o treinamento, não foi necessario imputar informações em valores nulos. Talvez seja necessario revisitar já que o dataset de teste pode ter dados nulos. 
2. A correlação entre as váriaveis de forma independente não tem uma boa correlação com o churn, então teremos que apostar em um conjunto de relações para tentar predizer o churn.
3. Existe uma quantidade satisfatoria de dados de cada categoria da target (churn) então técnicas de para equilibrar as classes não devem ser necessarias.
4. Tem poucas variaveis numericas e sem outliers explicitos
